# Data Visualization: State of the Industry 2021 - *2021 challenge*

In [49]:
# DATA ANALYSIS
import pandas as pd
import numpy as np

# TEXT ANALYSIS
import re # library for regular expression operations
import string # for string operations
# Stop words:
from nltk.corpus import stopwords  
stop_words = set(stopwords.words('english'))
# Tokenization:
from nltk.tokenize import word_tokenize
# Lemmatizer:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [50]:
# Load the data
df = pd.read_csv('../data/input/data_2021_jobtitles_taskstime_dvs-soti_v1.csv', low_memory=False)

In [51]:
df.head()

,JobTitle__lightlycleaned,RoleMultichoice_composite,TimeWorked,TimeDataPrep,TimeDataAnalysis,TimeIdeating,TimeProducingViz,TimeOtherVizTasks,OtherVizTasks__,altID
0,Data Visualization Developer,Journalist,30–39 hours,11–20 hours,6–10 hours,6–10 hours,11–20 hours,11–20 hours,"Writing, reporting and editing other people's ...",1
1,Data storyteller,"Leadership (Manager, Director, VP, etc.)",40–49 hours,5 hours or less,6–10 hours,11–20 hours,21–30 hours,6–10 hours,"Writing, blogging, social about the topic",2
2,Research & Data Analyst,Analyst,40–49 hours,5 hours or less,5 hours or less,5 hours or less,5 hours or less,5 hours or less,Writing takeaways associated with the visualiz...,3
3,"Director, Technical Consulting","Leadership (Manager, Director, VP, etc.)",40–49 hours,5 hours or less,5 hours or less,5 hours or less,5 hours or less,5 hours or less,Writing software to support making visualizations,4
4,Data Visualization Developer,Developer,50–59 hours,21–30 hours,6–10 hours,5 hours or less,5 hours or less,11–20 hours,Writing scripts to ingest new data,5


In [52]:
df.shape

(2165, 10)

In [53]:
df.RoleMultichoice_composite.value_counts(dropna=False)

Analyst                                     557
Leadership (Manager, Director, VP, etc.)    297
NaN                                         275
Designer                                    229
Developer                                   180
Scientist                                   122
Academic/Teacher                            114
Student                                     102
Engineer                                     98
None of these describes my role              95
Journalist                                   64
Teacher                                      21
Cartographer                                 11
Name: RoleMultichoice_composite, dtype: int64

In [54]:
# First assumption: nan job titles = None of these describes my role
# They both refer to something else, not listed in the job titles 
df['RoleMultichoice_composite'] = df['RoleMultichoice_composite'].fillna('None of these describes my role')
df.RoleMultichoice_composite.value_counts(dropna=False)

Analyst                                     557
None of these describes my role             370
Leadership (Manager, Director, VP, etc.)    297
Designer                                    229
Developer                                   180
Scientist                                   122
Academic/Teacher                            114
Student                                     102
Engineer                                     98
Journalist                                   64
Teacher                                      21
Cartographer                                 11
Name: RoleMultichoice_composite, dtype: int64

In [55]:
# Replacing remaining nan values with None
df = df.fillna('None')

In [56]:
df_roles = pd.DataFrame(df.RoleMultichoice_composite.value_counts(dropna=False)).reset_index()
df_roles.columns = ['role', 'persons_role']
df_roles

,role,persons_role
0,Analyst,557
1,None of these describes my role,370
2,"Leadership (Manager, Director, VP, etc.)",297
3,Designer,229
4,Developer,180
5,Scientist,122
6,Academic/Teacher,114
7,Student,102
8,Engineer,98
9,Journalist,64


In [57]:
df_roles.to_csv('../data/output/role_count.csv', index=False)

In [80]:
# Hours worked per week by role
week_hours_worked = pd.DataFrame(df.groupby(['RoleMultichoice_composite', 'TimeWorked'])['TimeWorked'].count())
week_hours_worked.columns = ['count']
week_hours_worked = week_hours_worked.reset_index()
week_hours_worked

,RoleMultichoice_composite,TimeWorked,count
0,Academic/Teacher,20–29 hours,3
1,Academic/Teacher,30–39 hours,28
2,Academic/Teacher,40–49 hours,47
3,Academic/Teacher,50–59 hours,19
4,Academic/Teacher,60–69 hours,6
...,...,...,...
76,Teacher,20–29 hours,1
77,Teacher,30–39 hours,7
78,Teacher,40–49 hours,7
79,Teacher,50–59 hours,3


In [81]:
# Adding a col with proportions within roles
week_hours_worked = (
    df_roles.set_index('role')
    .join(week_hours_worked.set_index('RoleMultichoice_composite'), on='role')
    .reset_index()
)
week_hours_worked['persons_perc'] = week_hours_worked['count']/week_hours_worked.persons_role

In [82]:
week_hours_worked.head()

,role,persons_role,TimeWorked,count,persons_perc
0,Analyst,557,20–29 hours,28,0.050269
1,Analyst,557,30–39 hours,148,0.265709
2,Analyst,557,40–49 hours,312,0.560144
3,Analyst,557,50–59 hours,39,0.070018
4,Analyst,557,60–69 hours,9,0.016158


In [83]:
week_hours_worked.to_csv('../data/output/week_hours_worked.csv', index=False)

In [14]:
# Columns about time tasks
time_var = ['TimeDataPrep', 'TimeDataAnalysis', 'TimeIdeating', 'TimeProducingViz', 'TimeOtherVizTasks']

In [15]:
# Count how many people spend a defined amount of time for every role and task (SMALL MULTIPLE PLOT)
df_list = []
for role in df.RoleMultichoice_composite.unique():
    for task in time_var:
        df_tmp = pd.DataFrame(df[df.RoleMultichoice_composite==role][task].value_counts(dropna=False)).reset_index()
        df_tmp.columns = ['hours', 'persons']
        df_tmp['task'] = task
        df_tmp['role'] = role
        df_list.append(df_tmp)
df_all = pd.concat(df_list, ignore_index=True)

In [16]:
df_all.head()

,hours,persons,task,role
0,5 hours or less,30,TimeDataPrep,Journalist
1,6–10 hours,16,TimeDataPrep,Journalist
2,11–20 hours,9,TimeDataPrep,Journalist
3,None,7,TimeDataPrep,Journalist
4,More than 30 hours,1,TimeDataPrep,Journalist


In [17]:
# dict to map hours' intervals to integer (order is important)
mapping_hours = {
    'None': 0,
    '5 hours or less': 1,
    '6–10 hours': 2,
    '11–20 hours': 3,
    '21–30 hours': 4,
    'More than 30 hours': 5
}
df_all['id_hours'] = df_all['hours'].map(mapping_hours)

In [18]:
# task -> integer (order is important)
mapping_task = {
    'TimeOtherVizTasks': 0,
    'TimeDataPrep': 1,
    'TimeDataAnalysis': 2,
    'TimeIdeating': 3,
    'TimeProducingViz': 4
}
df_all['id_task'] = df_all['task'].map(mapping_task)

In [19]:
# role -> integer (order is not important)
mapping_role = {
    'Journalist': 8,
    'Leadership (Manager, Director, VP, etc.)': 1,
    'Analyst': 0,
    'Developer': 3,
    'Teacher': 9,
    'Cartographer': 10,
    'Engineer': 7,
    'Student': 6,
    'Academic/Teacher': 5,
    'Designer': 2,
    'Scientist': 4,
    'None of these describes my role': 11,
}
df_all['id_role'] = df_all['role'].map(mapping_role)

In [20]:
df_all.head()

,hours,persons,task,role,id_hours,id_task,id_role
0,5 hours or less,30,TimeDataPrep,Journalist,1,1,8
1,6–10 hours,16,TimeDataPrep,Journalist,2,1,8
2,11–20 hours,9,TimeDataPrep,Journalist,3,1,8
3,None,7,TimeDataPrep,Journalist,0,1,8
4,More than 30 hours,1,TimeDataPrep,Journalist,5,1,8


In [21]:
# Adding a col with proportions within roles
df_all = df_roles.set_index('role').join(df_all.set_index('role'), on='role').reset_index()
df_all['persons_perc'] = df_all.persons/df_all.persons_role

In [22]:
df_all.head()

,role,persons_role,hours,persons,task,id_hours,id_task,id_role,persons_perc
0,Analyst,557,5 hours or less,223,TimeDataPrep,1,1,0,0.400359
1,Analyst,557,6–10 hours,161,TimeDataPrep,2,1,0,0.289048
2,Analyst,557,11–20 hours,107,TimeDataPrep,3,1,0,0.192101
3,Analyst,557,None,34,TimeDataPrep,0,1,0,0.061041
4,Analyst,557,21–30 hours,31,TimeDataPrep,4,1,0,0.055655


In [23]:
df_all.to_csv('../data/output/time_spent_task.csv', index=False)

## Avg. time spent overall - comparison between job titles

In [24]:
# How many hours on average each role spend on each task?
# assuming equally spaced weights for the different hours' ranges
w_avg = []
for role in df_all.role.unique():
    for task in df_all.task.unique():
        df_tmp = df_all[(df_all.role==role) & (df_all.task==task)]
        weighted_av = np.dot(df_tmp.persons_perc, df_tmp.id_hours)
        w_avg.append((task, role, weighted_av/15)) # 15 = sum of weights (0+1+2+3+4+5)

In [25]:
df_w_avg = pd.DataFrame(w_avg)
df_w_avg.columns = ['task', 'role', 'w_avg']

In [26]:
df_w_avg.sort_values(by=['role', 'w_avg']).head()

,task,role,w_avg
34,TimeOtherVizTasks,Academic/Teacher,0.058480
32,TimeIdeating,Academic/Teacher,0.067251
30,TimeDataPrep,Academic/Teacher,0.090643
33,TimeProducingViz,Academic/Teacher,0.092398
31,TimeDataAnalysis,Academic/Teacher,0.108772


In [28]:
df_w_avg['task_id'] = df_w_avg['task'].map(mapping_task)
df_w_avg.to_csv('../data/output/weighted_avg.csv', index=False)

# Text analysis: understanding the *other tasks*

In [84]:
df[df.OtherVizTasks__=='None'].shape # 1450 erano nan

(1453, 10)

In [85]:
n=df.shape[0]
n, n-1453, (n-1453)/n

(2165, 712, 0.3288683602771363)

In [86]:
# Proportion of people talking about other tasks
df_roles_other_task = df[df.OtherVizTasks__!='None'].RoleMultichoice_composite.value_counts().reset_index()
df_roles_other_task.columns = ['role', 'persons_role_other']
df_roles_other_task = df_roles.set_index('role').join(df_roles_other_task.set_index('role'))
df_roles_other_task['persons_prop_other'] = df_roles_other_task.persons_role_other/df_roles_other_task.persons_role
df_roles_other_task.sort_values('persons_prop_other')

,persons_role,persons_role_other,persons_prop_other
role,,,
Student,102,24,0.235294
Analyst,557,138,0.247756
None of these describes my role,370,99,0.267568
Cartographer,11,3,0.272727
Scientist,122,35,0.286885
Academic/Teacher,114,36,0.315789
Journalist,64,22,0.343750
Developer,180,68,0.377778
"Leadership (Manager, Director, VP, etc.)",297,125,0.420875


In [87]:
# Analysis of the field
other_tasks = pd.DataFrame(df[df.OtherVizTasks__!='None'].OtherVizTasks__)
other_tasks

,OtherVizTasks__
0,"Writing, reporting and editing other people's ..."
1,"Writing, blogging, social about the topic"
2,Writing takeaways associated with the visualiz...
3,Writing software to support making visualizations
4,Writing scripts to ingest new data
...,...
713,xys
714,30
715,5
716,3


In [33]:
# Cleaning steps are performed as for the order below:
# 1. lowercasing
# 2. punctuation removal
# 3. remove multiple spaces
# 4. remove spaces at the beginning and at the end of each text
# 5. Tokenization
# 6. Stop words removal
# 7. Lemmatization (both on verbs and nouns)
def cleaning_text(text):
    text = text.lower()
    for el in string.punctuation:
        text = re.sub(f'\\{el}', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text_tokens = word_tokenize(text)
    text_tokens_stop_words = [i for i in text_tokens if not i in stop_words]
    text_tokens_lemm = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in text_tokens_stop_words]
    text_tokens_lemm = [wordnet_lemmatizer.lemmatize(word, pos='n') for word in text_tokens_lemm]
    return text_tokens_lemm

In [34]:
# Function to join tokens and create bigrams
def join_from_onegrams(onegram_text, true_bigr_set, sep='_'):
    res = []
    skip = False
    for prev, curr in zip(onegram_text[:-1], onegram_text[1:]):
        if skip:
            skip = False
            continue
        if f'{prev}_{curr}' in true_bigr_set:
            res.append(f'{prev}{sep}{curr}')
            skip = True
        else:
            res.append(prev)
    if onegram_text[1:] and not skip:
        res.append(onegram_text[-1])
    if onegram_text and not onegram_text[1:]:
        res = onegram_text
    return res

In [35]:
bigrams = ['data_collection', 'ux_design', 'web_design', 'data_model', 'data_source']

In [36]:
df['text_cleaned'] = df.OtherVizTasks__.apply(lambda x: cleaning_text(str(x)))
df['text_cleaned'] = (
    df.text_cleaned.apply(lambda x: join_from_onegrams(x,  true_bigr_set=bigrams))
)
df['text_cleaned_txt'] = df.text_cleaned.apply(lambda x: ' '.join(x))
df['text_cleaned_txt'] = df.text_cleaned_txt.apply(lambda x: '' if x=='none' else x)
df['text_cleaned'] = df.text_cleaned_txt.apply(lambda x: x.split(' ') if x!='' else [])

In [37]:
# create some clusters of related words / topics
topics = {
    'client': ['stakeholder', 'client', 'customer'],
    'coding': ['code', 'script', 'software', 'tool', 'library', 'development'],
    'blogging': ['blog', 'blogging', 'social'],
    'presentation': ['slide', 'present', 'ppt', 'ppts', 'presentation', 'communicate', 'communication'],
    'feedback': ['feedback', 'review'],
    'teaching': ['teach', 'train', 'workshop', 'instruction', 'instruction', 'course',
                 'instructional', 'lecture', 'class', 'education', 'educational', 'coach'],
    'research': ['research', 'learn', 'study'],
    'documenting': ['document', 'documentation', 'report'],
    'dashboard': ['dashboard'],
    'style_guide': ['style'],
    'meeting': ['meet'],
    'management': ['management', 'manage', 'plan', 'strategy', 'strategic'],
    'web_design': ['ux_design', 'web_design'],
    'data_collection': ['data_collection', 'collect', 'etl', 'data_model', 'requirement', 'data_source'],
    'accessibility': ['accessibility']
}

In [38]:
def get_tag(txt):
    return [key for key, val in topics.items() if len((set(txt) & set(val)))>0]

# Esempio
test_txt = ['code', 'style', 'social_media', 'project_management']
#test_txt = []
get_tag(test_txt)

['coding', 'style_guide']

In [39]:
df['tag'] = df.text_cleaned.apply(lambda x: get_tag(x))

In [40]:
# Check residuals
residuals = df[(df.tag.apply(lambda x: len(x)==0)) & (df.text_cleaned.apply(lambda x: len(x)>0))]
pd.Series([el for el in residuals.text_cleaned for el in el]).value_counts().head(20)
# residuals[residuals.text_cleaned_txt.str.contains('plan')]

data             70
visualization    33
design           13
viz              12
write            11
others           10
create            9
visualisation     8
user              8
work              7
use               7
team              6
task              6
analysis          6
make              5
etc               5
look              5
find              5
support           5
best              5
dtype: int64

In [41]:
residuals.shape

(239, 13)

In [42]:
all_count = []
for el in topics:
    count_t = df[df.tag.apply(lambda x: el in x)].shape[0]
    all_count.append((el, count_t))
df_tag_all_count = pd.DataFrame(all_count)
df_tag_all_count.columns = ['tag', 'tag_total_n']
df_tag_all_count.sort_values(by='tag_total_n')

,tag,tag_total_n
12,web_design,5
14,accessibility,5
2,blogging,9
9,style_guide,9
8,dashboard,17
7,documenting,36
4,feedback,41
13,data_collection,43
10,meeting,50
0,client,56


In [43]:
df_tag = pd.DataFrame(df.tag.value_counts()).reset_index()
df_tag.columns = ['tag', 'n_tag']
df_tag['len_tag'] = df_tag.tag.apply(lambda x: len(x))
df_tag

,tag,n_tag,len_tag
0,[],1695,0
1,[teaching],47,1
2,[research],46,1
3,[coding],43,1
4,[management],39,1
...,...,...,...
77,"[presentation, dashboard, style_guide, meeting...",1,5
78,"[presentation, web_design]",1,2
79,"[teaching, meeting, management]",1,3
80,"[feedback, teaching, meeting]",1,3


In [44]:
df_tag.len_tag.value_counts()

2    43
3    20
1    15
5     2
0     1
4     1
Name: len_tag, dtype: int64

In [45]:
df_tag_1 = df_tag[df_tag.len_tag==1]
df_tag_1['tag'] = df_tag_1.tag.apply(lambda x: x[0])
df_tag_1.set_index('tag').join(df_tag_all_count.set_index('tag'), on='tag').reset_index()

/var/folders/1j/zhd4kg153k3dj_kpv7twm9gr67y1mk/T/ipykernel_95485/2195395591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tag_1['tag'] = df_tag_1.tag.apply(lambda x: x[0])


,tag,n_tag,len_tag,tag_total_n
0,teaching,47,1,75
1,research,46,1,69
2,coding,43,1,70
3,management,39,1,75
4,presentation,38,1,64
5,meeting,27,1,50
6,data_collection,25,1,43
7,client,21,1,56
8,documenting,16,1,36
9,feedback,16,1,41


In [46]:
df_tag_1.to_csv('../data/output/tag1.csv')

In [47]:
df_tag_2 = df_tag[df_tag.len_tag==2].reset_index(drop=True)
df_tag_2.head()

,tag,n_tag,len_tag
0,"[client, management]",7,2
1,"[teaching, management]",6,2
2,"[coding, research]",5,2
3,"[coding, documenting]",5,2
4,"[coding, teaching]",4,2


In [48]:
(
    df_tag_2
    .join(pd.DataFrame(df_tag_2['tag'].to_list(), columns=['tag1','tag2']))
    .drop('tag', axis=1)
    .to_csv('../data/output/tag12.csv')
)